## Import & Load Dataset

In [ ]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from google.colab import drive
import glob
import json
import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import PreTrainedTokenizerFast, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from torch import nn
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import PPOTrainer, PPOConfig

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/MyDrive/멋사자 AI 2차 실전 프로젝트/한국어 성능이 개선된 초거대AI 언어모델 개발 및 데이터/Training/02.라벨링데이터/TL_02.RLHF데이터'

if os.path.exists(file_path):
  print('yes')

yes


In [ ]:
file_name1 = file_path + '/RMlabel.json'
file_name2 = file_path + '/SFTlabel.json'
file_name3 = file_path + '/PPOlabel.json'

with open(file_name1, 'r') as file:
  data = json.load(file)
  data1 = data['data_info']

with open(file_name2, 'r') as file:
  data = json.load(file)
  data2 = data['data_info']

with open(file_name3, 'r') as file:
  data = json.load(file)
  data3 = data['data_info']

In [ ]:
SFTdf = pd.json_normalize(data2)
RMdf = pd.json_normalize(data1)
PPOdf = pd.json_normalize(data3)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
SFTdf.head(5)

,data_id,question_type,question_count,question,data_category.main,data_category.middle,answer.answer_count,answer.contents
0,04779be1-67ac-4b70-8dc2-fda6dc353b2a,생성,6,화학공학이 다른 공학 분야와 어떻게 다른가요?,문어체,공학,198,화학공학은 화학과 공학의 합성으로 이루어진 학문 분야입니다. 다른 일반적인 공학 분...
1,cb6c0bcf-61c5-4a40-a0ef-b5090b3bac4c,생성,10,정제 탄수화물과 비정제 탄수화물은 사람의 소화 과정에 어떤 차이점을 보이는가?,문어체,보건,107,정제 탄수화물은 실효성이 높아 소화 과정에서 빠르게 분해되고 흡수됩니다. 이는 혈당...
2,701a4cf9-28ad-4d67-ae35-05963b14683b,생성,6,사상의학에서 유전자 분석은 어떻게 활용되고 있나요?,구어체,보건,229,사상의학에서 유전자 분석은 다양한 방법으로 활용되고 있습니다. 일반적으로 질병 예방...
3,832d1fe0-42a0-4707-9367-25b13477eee0,생성,8,도자기를 고르게 돌리기 위해 사용하는 도구의 이름은?,구어체,예체능,20,도자기를 고르게 돌리기 위해 사용하는 도구는 '도마'입니다. 도마는 도자기를 고르게...
4,ca7b24aa-17c4-4d22-872e-86710bd023ee,생성,7,동물 질병 예방을 위한 접종을 어떻게 진행하는가?,구어체,보건,208,동물 질병 예방을 위한 접종은 동물의 종류와 질병에 따라 다소 차이가 있을 수 있지...


In [ ]:
trainQA_df = SFTdf[['question','answer.contents']]

In [ ]:
trainQA_df.head(5)

,question,answer.contents
0,화학공학이 다른 공학 분야와 어떻게 다른가요?,화학공학은 화학과 공학의 합성으로 이루어진 학문 분야입니다. 다른 일반적인 공학 분...
1,정제 탄수화물과 비정제 탄수화물은 사람의 소화 과정에 어떤 차이점을 보이는가?,정제 탄수화물은 실효성이 높아 소화 과정에서 빠르게 분해되고 흡수됩니다. 이는 혈당...
2,사상의학에서 유전자 분석은 어떻게 활용되고 있나요?,사상의학에서 유전자 분석은 다양한 방법으로 활용되고 있습니다. 일반적으로 질병 예방...
3,도자기를 고르게 돌리기 위해 사용하는 도구의 이름은?,도자기를 고르게 돌리기 위해 사용하는 도구는 '도마'입니다. 도마는 도자기를 고르게...
4,동물 질병 예방을 위한 접종을 어떻게 진행하는가?,동물 질병 예방을 위한 접종은 동물의 종류와 질병에 따라 다소 차이가 있을 수 있지...


## QA Model train

In [ ]:
trainQA_df, testQA_df = train_test_split(trainQA_df, test_size=0.2, random_state=42)

In [ ]:
model_name = 'skt/kogpt2-base-v2'

tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
bos_token='</s>', eos_token='</s>', unk_token='<unk>',
pad_token='<pad>', mask_token='<mask>')
model = GPT2LMHeadModel.from_pretrained(model_name)
model.to(device)

tokenizer.json:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [ ]:
def preprocess_data(df):

  input_text = f"Question: {df['question]']}"
  target_text = df['answer.contents']

  input_encoding = tokenizer(input_text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
  target_encoding = tokenizer(target_text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

  input_ids = input_encoding['input_ids'].squeeze()
  labels = target_encoding['input_ids'].squeeze()

  return {'input_ids': input_ids, 'labels': labels}

In [ ]:
class QADataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        question = self.df.iloc[idx]['question']
        answer = self.df.iloc[idx]['answer.contents']
        input_text = f"Question: {question} Answer: {answer}"
        # 토크나이징 후 텐서로 변환
        encoding = self.tokenizer(input_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return encoding

In [ ]:
train_dataset =QADataset(trainQA_df, tokenizer)
test_dataset = QADataset(testQA_df, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [ ]:
def train(model, train_loader, optimizer, num_epochs=3):
    model.train()

    for epoch in range(num_epochs):
        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
            # 입력 텍스트와 라벨을 GPU로 이동
            input_ids = batch['input_ids'].squeeze(1).to(device)
            attention_mask = batch['attention_mask'].squeeze(1).to(device)
            labels = input_ids

            # 모델을 통해 예측 생성
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        tqdm.write(f"Loss: {loss.item()}")

# 평가 함수 정의
def evaluate(model, test_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating", leave=False):
            input_ids = batch['input_ids'].squeeze(1).to(device)
            attention_mask = batch['attention_mask'].squeeze(1).to(device)
            labels = input_ids

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()

    avg_loss = total_loss / len(test_loader)
    print(f"Test Loss: {avg_loss}")

In [ ]:
# 옵티마이저 정의
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-6)

# 모델 학습
train(model, train_loader, optimizer, num_epochs=50)

# 테스트 데이터로 모델 평가
evaluate(model, test_loader)

model.save_pretrained('./drive/MyDrive/멋사/챗봇프로젝트/QAmodel')
tokenizer.save_pretrained('./drive/MyDrive/멋사/챗봇프로젝트/QAmodel')

Loss: 1.2993544340133667


Test Loss: 1.1914691119991765


('./drive/MyDrive/멋사/챗봇프로젝트/QAmodel/tokenizer_config.json',
 './drive/MyDrive/멋사/챗봇프로젝트/QAmodel/special_tokens_map.json',
 './drive/MyDrive/멋사/챗봇프로젝트/QAmodel/tokenizer.json')

## Generate Answer (QA)

In [ ]:
def generate_answer(model, tokenizer, testQA_df):
  model.eval()

  input_text = testQA_df.iloc[2]['question']
  print(input_text)

  input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

  output = model.generate(
      input_ids=input_ids,
      max_length=128,   # 생성될 텍스트의 최대 길이
      num_beams=5,      # Beam Search 사용 (다양한 후보 생성)
      no_repeat_ngram_size=2,  # 반복되는 n-gram 방지
      temperature=0.7,  # 더 확정적인 출력을 위해 낮은 값 사용
      top_p=0.9,        # nucleus sampling을 통해 확률 분포 조정
      top_k=50,         # 상위 k개 토큰만 선택
      pad_token_id=tokenizer.eos_token_id,  # EOS 토큰으로 패딩
      early_stopping=True  # 답변이 끝날 때까지 기다리도록 설정
  )

  generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

  # 출력된 답변
  print("Generated Answer: ", generated_text)

In [ ]:
model2 = GPT2LMHeadModel.from_pretrained('./drive/MyDrive/멋사/챗봇프로젝트/QAmodel50epochs')
tokenizer2 = PreTrainedTokenizerFast.from_pretrained('./drive/MyDrive/멋사/챗봇프로젝트/QAmodel50epochs')

In [ ]:
generate_answer(model, tokenizer, testQA_df)

에너지 효율적인 시스템과 장비는 어떻게 설치되고 관리되는가?
Generated Answer:  에너지 효율적인 시스템과 장비는 어떻게 설치되고 관리되는가?’ 라는 질문에 대한 답변이다.
이러한 질문에 답하기 위해선 먼저 ‘스마트홈’ 구축의 필요성부터 살펴봐야 한다.
우선 스마트홈은 가정 내 모든 가전기기를 제어하고 제어할 수 있는 시스템으로 가정내 모든 가전제품을 제어하는 것은 물론 에너지 사용량을 실시간으로 모니터링 할 수 있어 에너지 효율을 극대화할 수 있다.
또한 스마트폰을 통해 언제 어디서든 스마트폰으로 집안의 가전제품 상태를 실시간 확인할 수 있기 때문에 에너지 절약에 대한 걱정을 덜 수 있을 뿐만 아니라 에너지 절감 효과도 기대할 수 있다.


In [ ]:
model2.to(device)
generate_answer(model2, tokenizer2, testQA_df)

에너지 효율적인 시스템과 장비는 어떻게 설치되고 관리되는가?
Generated Answer:  에너지 효율적인 시스템과 장비는 어떻게 설치되고 관리되는가? 답변: 효율적인 시스템을 설치하고 관리하기 위해 다음과 같은 단계를 따를 수 있습니다:

1. 목표 설정: 시스템의 목표를 설정하고 해당 목표를 달성하기 위한 계획을 수립합니다. 이 단계에서는 시스템의 우선순위를 결정하여 문제를 해결하고 성능을 향상시킬 수 있도록 합니다.

2. 자원 할당: 자원을 할당하고 필요한 자원은 할당합니다. 이를 통해 시스템의 자원을 효율적으로 사용할 수 있습니다. 예를 들어, 웹 브라우저, 소셜 미디어 애플리케이션, 이메일 등 자원 할당을 할 수 있습니다.

3. 프로세스 최적화: 웹사이트의 구조, 콘텐츠, 링크 등의 프로세스를 최적화합니다. 이를 위해 키워


## RM Model train

In [ ]:
RMdf.head(5)

,data_id,question_type,question_count,question,data_category.main,data_category.middle,answer01.answer_count,answer01.ranking,answer01.contents,answer02.answer_count,...,answer02.contents,answer03.answer_count,answer03.ranking,answer03.contents,answer04.answer_count,answer04.ranking,answer04.contents,answer05.answer_count,answer05.ranking,answer05.contents
0,0827c2bf-592d-4048-8f95-3c5fe336c1d1,생성,8,동물에 대한 인식을 높이고 교육하는 데 관심 있어?,문어체,자연,42,1,"네, 저는 동물에 대한 인식을 높이고 교육하는 데 관심이 있습니다. 어떤 종류의 동...",35,...,"네, 저는 동물에 대한 인식을 높이고 교육하는 데 관심이 있습니다. 어떤 도움이 필...",27,3,"네, 저는 동물에 대한 인식과 교육을 증진하는 것에 관심이 있습니다. 동물에 대한 ...",13,4,"네, 저는 동물에 대한 인식을 높이고 교육하는 데 관심이 있습니다. 어떤 도움이 필...",13,5,"네, 저는 동물에 대한 인식과 교육을 돕기 위해 노력하고 있습니다. 어떤 도움이 필..."
1,e14a45b5-9e2c-44a1-ba37-e1015116e665,생성,5,언론이 사회에 미치는 영향은 어때?,문어체,사회,140,1,언론은 사회에 큰 영향을 미칩니다. 언론은 사회의 다양한 이슈를 보도하고 정보를 제...,132,...,"언론은 사회에 많은 영향을 미칩니다. 먼저, 언론은 정보를 전달하고 사회 이슈에 대...",112,3,"언론은 사회에 많은 영향을 미칩니다. 우선, 언론은 정보를 전달하고 사회 이슈를 보...",95,4,언론은 사회에 많은 영향을 미칩니다. 언론은 정보를 전달하고 사회의 이슈를 보도함으...,62,5,언론은 사회에 많은 영향을 미칩니다. 언론은 정보를 전달하고 의견을 형성하는데 큰 ...
2,6a5d63bf-ca9e-46a9-8e5c-0d6c5ea7ce81,생성,6,일본 항공권 예약하는 방법 알려 줘.,구어체,산업,158,1,일본 항공권을 예약하는 방법은 다음과 같습니다:\n\n1. 인터넷 검색 엔진을 사용...,144,...,일본 항공권을 예약하는 방법은 여러 가지가 있습니다. 가장 간단한 방법은 온라인으로...,156,3,일본 항공권을 예약하는 방법은 여러 가지가 있습니다. 다음은 몇 가지 일반적인 예약...,150,4,일본 항공권을 예약하는 방법은 여러 가지가 있습니다. 아래에 몇 가지 일반적인 예약...,125,5,일본 항공권을 예약하는 방법은 여러 가지가 있습니다. 다음은 몇 가지 일반적인 방법...
3,c6c9ba7c-116d-4b07-8815-00e0826044ad,생성,4,노래를 배워본 적 있어?,구어체,예체능,19,1,"아니요, 저는 노래를 배울 수는 없지만 노래에 대한 정보나 도움을 드릴 수 있습니다...",18,...,"아니요, 저는 노래를 배울 수는 없지만, 노래를 배우는 데 도움이 되는 팁이나 리소...",16,3,"아니요, 저는 노래를 배우거나 부를 수 없습니다. 하지만 노래에 대해 어떤 도움이 ...",16,4,"아니요, 저는 노래를 배울 수는 없지만, 노래에 대한 정보나 조언을 제공할 수 있습...",12,5,저는 노래를 배우는 데 도움이 될 수 있습니다. 어떤 노래를 배우고 싶으신가요?
4,081afbe1-8324-4650-9fc2-e5987f37cd1e,생성,5,홈트레이닝 운동 영상을 추천해 줘.,구어체,예체능,64,1,홈트레이닝 운동을 위한 영상을 추천해 드리겠습니다. 아래의 영상들은 다양한 운동을 ...,62,...,홈트레이닝 운동 영상을 추천해 드리겠습니다. 아래의 영상들은 다양한 운동을 집에서 ...,82,3,홈트레이닝 운동을 위한 다양한 영상을 추천해 드리겠습니다. 아래의 링크를 클릭하면 ...,81,4,확실히 홈트레이닝 운동은 편리하면서도 효과적인 방법이죠. 여러 가지 운동 영상 중 ...,60,5,홈트레이닝 운동 영상을 추천해 드리겠습니다. 아래의 운동 영상들을 참고해 보세요:\...


In [ ]:
RMdf['answer01.contents'][5]

'학교에서 배운 내용은 영화 제작과 관련된 다양한 측면에 적용될 수 있습니다. \n\n1. 시나리오 작성: 학교에서 배운 문학이나 글쓰기 수업에서 얻은 글쓰기 기법이나 구조적인 요소들은 영화 시나리오 작성에 큰 도움이 될 수 있습니다. 시나리오는 영화의 기획 단계에서 중요한 역할을 하며, 흥미로운 이야기를 구성하는 데 학교에서 배운 내용들을 적용할 수 있습니다.\n\n2. 연출 및 촬영: 미술, 사진, 영상 등의 수업에서 배운 시각적인 원리와 기법들은 영화의 연출과 촬영 단계에서 적용될 수 있습니다. 예를 들어, 학교에서 배운 조명, 각도, 프레이밍 등의 요소들은 영화의 시각적인 효과를 극대화하는 데 도움이 됩니다.\n\n3. 편집: 학교에서 배운 문학이나 영화 이론 수업에서 얻은 지식들은 영화의 편집 단계에서도 활용될 수 있습니다. 영화의 편집은 이야기의 전달과 감정의 전달에 큰 영향을 미치며, 학교에서 배운 내용들을 활용하여 효과적인 편집을 할 수 있습니다.\n\n4. 음향: 음악, 사운드 디자인 등의 수업에서 배운 내용들은 영화의 음향 부분에서 적용될 수 있습니다. 학교에서 배운 음악적 원리나 사운드 효과들은 영화의 분위기와 감정을 전달하는 데 큰 역할을 합니다.\n\n5. 주제와 메시지: 학교에서 배운 문학 작품이나 사회과학 수업에서 다룬 주제와 메시지들은 영화의 스토리텔링과 메시지 전달에 활용될 수 있습니다. 영화는 사회적인 이슈나 인간의 경험을 다룰 수 있으며, 학교에서 배운 내용들을 통해 영화의 내용과 메시지를 강화시킬 수 있습니다.\n\n이처럼 학교에서 배운 내용들은 영화 제작의 다양한 측면에 적용될 수 있으며, 풍부한 지식과 창의성을 발휘하여 좋은 영화를 만들 수 있습니다.'

In [ ]:
class RewardModel(nn.Module):
  def __init__(self, model_name):
    super(RewardModel, self).__init__()
    # fine-tuning된 SFT 모델 불러오기
    self.base_model = AutoModel.from_pretrained(model_name)
    # 점수 출력 (스칼라 값)
    self.reward_head = nn.Linear(self.base_model.config.hidden_size, 1)

  def forward(self, input_ids, attention_mask):
    # 기본 모델의 출력
    outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
    # [CLS] 토큰을 사용하여 점수 예측
    logits = self.reward_head(outputs.last_hidden_state[:, 0, :])
    return logits.squeeze(-1)

In [ ]:
tokenizer2.model_max_length=512

In [ ]:
# 모델의 최대 입력 길이 확인
print(tokenizer2.model_max_length)

512


In [ ]:
class RMDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=512):
        self.inputs = []
        self.scores = []
        for _, row in data.iterrows():
            prompt = row['question']
            responses = [row[f'answer0{i+1}.contents'] for i in range(3)]
            scores = [row[f'answer0{i+1}.ranking'] for i in range(3)]

            # 응답을 각각 토큰화하고 입력 리스트와 스코어 저장
            for response, score in zip(responses, scores):
                tokenized = tokenizer(
                    f"{prompt} {response}",
                    return_tensors="pt",
                    padding='max_length',
                    truncation=True,
                    max_length=max_length
                )
                self.inputs.append({
                    'input_ids': tokenized['input_ids'].squeeze(0),
                    'attention_mask': tokenized['attention_mask'].squeeze(0)
                })
                self.scores.append(score)

    def __len__(self):
        return len(self.scores)

    def __getitem__(self, idx):
        return {
            'input_ids': self.inputs[idx]['input_ids'],
            'attention_mask': self.inputs[idx]['attention_mask'],
            'score': torch.tensor(self.scores[idx], dtype=torch.float32)
        }

In [ ]:
trainRM_df, testRM_df = train_test_split(RMdf, test_size=0.2, random_state=42)

In [ ]:
rm_dataset = RMDataset(trainRM_df, tokenizer2)
dataloader = DataLoader(rm_dataset, batch_size=8, shuffle=True)
val_dataset = RMDataset(testRM_df, tokenizer2)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# 체크포인트 저장 함수
def save_checkpoint(model, optimizer, epoch, loss, path):
    checkpoint = {
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': loss
    }
    torch.save(checkpoint, path)
    print(f"Checkpoint saved at {path}")

# 체크포인트 로드 함수
def load_checkpoint(path, model, optimizer):
    if os.path.exists(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        loss = checkpoint['loss']
        print(f"Checkpoint loaded. Resuming from epoch {start_epoch}, loss: {loss}")
        return start_epoch, loss
    else:
        print("No checkpoint found. Starting from scratch.")
        return 0, None


In [ ]:
loss_fn = nn.MarginRankingLoss(margin=1.0)
# 체크포인트 경로 설정
checkpoint_path = './drive/MyDrive/멋사/챗봇프로젝트/RMmodel_checkpoint.pt'

# 모델과 옵티마이저 초기화
model = RewardModel('./drive/MyDrive/멋사/챗봇프로젝트/QAmodel50epochs').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# 체크포인트 로드
start_epoch, prev_loss = load_checkpoint(checkpoint_path, model, optimizer)

num_epochs = 10
train_losses = []
val_losses = []

for epoch in range(start_epoch, num_epochs):
    model.train()
    epoch_train_loss = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} Training", leave=False):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        scores = batch['score'].to(device)

        optimizer.zero_grad()

        # 점수 예측
        predicted_scores = model(input_ids=input_ids, attention_mask=attention_mask)

        # Pairwise 손실 계산
        true_diff = (scores[:, None] - scores[None, :]).flatten()
        predicted_diff = (predicted_scores[:, None] - predicted_scores[None, :]).flatten()

        mask = true_diff != 0
        true_diff = true_diff[mask]
        predicted_diff = predicted_diff[mask]

        target = torch.sign(true_diff)  # 순위 차이에 따라 +1, -1 설정

        loss = loss_fn(predicted_diff, true_diff, target)

        loss.backward()
        optimizer.step()

        epoch_train_loss += loss.item()

    avg_train_loss = epoch_train_loss / len(dataloader)
    train_losses.append(avg_train_loss)
    print(f"Epoch {epoch + 1}/{num_epochs} - Train Loss: {avg_train_loss:.4f}")

    # Validation Loop
    model.eval()
    epoch_val_loss = 0
    with torch.no_grad():
        for batch in tqdm(val_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} Validation", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            scores = batch['score'].to(device)

            # 점수 예측
            predicted_scores = model(input_ids=input_ids, attention_mask=attention_mask)

            # Pairwise 손실 계산
            true_diff = (scores[:, None] - scores[None, :]).flatten()
            predicted_diff = (predicted_scores[:, None] - predicted_scores[None, :]).flatten()

            mask = true_diff != 0
            true_diff = true_diff[mask]
            predicted_diff = predicted_diff[mask]

            target = torch.sign(true_diff)  # 순위 차이에 따라 +1, -1 설정

            val_loss = loss_fn(predicted_diff, target.float())
            epoch_val_loss += val_loss.item()

        avg_val_loss = epoch_val_loss / len(val_dataloader)
        val_losses.append(avg_val_loss)
        print(f"Epoch {epoch + 1}/{num_epochs} - Validation Loss: {avg_val_loss:.4f}")

    # 체크포인트 저장
    save_checkpoint(model, optimizer, epoch, avg_train_loss, checkpoint_path)

# 최종 모델 저장
model_save_path = './drive/MyDrive/멋사/챗봇프로젝트/RMmodel'
torch.save(model.state_dict(), model_save_path)
print(f"Final model saved to {model_save_path}")


No checkpoint found. Starting from scratch.


Epoch 1/10 - Train Loss: 1.7797


Epoch 1/10 - Validation Loss: 1.8750
Checkpoint saved at ./drive/MyDrive/멋사/챗봇프로젝트/RMmodel_checkpoint.pt


Epoch 2/10 - Train Loss: 1.7792


Epoch 2/10 - Validation Loss: 1.8750
Checkpoint saved at ./drive/MyDrive/멋사/챗봇프로젝트/RMmodel_checkpoint.pt


...

Epoch 7/10 - Train Loss: 1.7804


Epoch 7/10 - Validation Loss: 1.8750
Checkpoint saved at ./drive/MyDrive/멋사/챗봇프로젝트/RMmodel_checkpoint.pt


Epoch 8/10 - Train Loss: 1.7742


Epoch 8/10 - Validation Loss: 1.8750
Checkpoint saved at ./drive/MyDrive/멋사/챗봇프로젝트/RMmodel_checkpoint.pt


Epoch 9/10 Training:  54%|█████▍    | 4282/7923 [28:25<24:16,  2.50it/s]

In [ ]:
def generate_scores(model, prompt, responses):
    model.eval()
    tokenized_responses = [
        tokenizer(f"{prompt} {response}", return_tensors="pt", padding=True, truncation=True)
        for response in responses
    ]

    predicted_scores = []
    with torch.no_grad():
        for response in tokenized_responses:
            input_ids = response['input_ids'].squeeze(1).to(device)
            attention_mask = response['attention_mask'].squeeze(1).to(device)
            predicted_score = model(input_ids=input_ids, attention_mask=attention_mask)
            predicted_scores.append(predicted_score.item())

    return predicted_scores


## Generate Answer (QA+RM)

In [ ]:
def generate_answer(generate_model, rank_model, tokenizer, testQA_df):
  generate_model.eval()
  rank_model.eval()

  input_text = testQA_df.iloc[2]['question']
  print(input_text)

  input_ids = tokenizer.encode(input_text, return_tensors='pt').to(device)

  outputs = generate_model.generate(
      input_ids=input_ids,
      max_length=128,   # 생성될 텍스트의 최대 길이
      do_sample=True, # Sampling
      # num_beams=3, # Beam Search 사용 (다양한 후보 생성)
      num_return_sequences=3, # 3개의 응답 생성
      no_repeat_ngram_size=2,  # 반복되는 n-gram 방지
      temperature=0.7,  # 더 확정적인 출력을 위해 낮은 값 사용
      top_p=0.9,        # nucleus sampling을 통해 확률 분포 조정
      top_k=50,         # 상위 k개 토큰만 선택
      pad_token_id=tokenizer.eos_token_id,  # EOS 토큰으로 패딩
      early_stopping=True  # 답변이 끝날 때까지 기다리도록 설정
  )

  generated_text = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

  # 출력된 3개의 응답
  for i, response in enumerate(generated_text):
    print(f"Response {i+1}: {response}")
    print('-'*100)

  rank_scores = []
  for response in generated_text:
      input_ids = tokenizer(f"{input_text} {response}", return_tensors="pt", padding=True, truncation=True).input_ids.to(device)
      attention_mask = tokenizer(f"{input_text} {response}", return_tensors="pt", padding=True, truncation=True).attention_mask.to(device)

      with torch.no_grad():
          score = rank_model(input_ids=input_ids, attention_mask=attention_mask)
          rank_scores.append(score.item())

  # 점수에 따라 답변 정렬
  ranked_responses = sorted(zip(generated_text, rank_scores), key=lambda x: x[1], reverse=True)

  # 최적의 답변 출력
  best_response = ranked_responses[0][0]
  print("Best Response:", best_response)

  return ranked_responses, best_response

In [ ]:
checkpoint_path = './drive/MyDrive/멋사/챗봇프로젝트/RMmodel_checkpoint.pt'
model = RewardModel('./drive/MyDrive/멋사/챗봇프로젝트/QAmodel50epochs').to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

# 체크포인트 로드 (학습 상태 복원 필요 없음)
_, _ = load_checkpoint(checkpoint_path, model, optimizer)

<ipython-input-24-cbc36f05de00>:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


Checkpoint loaded. Resuming from epoch 10, loss: 1.7776841481165588


In [ ]:
ranked, best = generate_answer(model2, model, tokenizer2, testQA_df)

에너지 효율적인 시스템과 장비는 어떻게 설치되고 관리되는가?


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Response 1: 에너지 효율적인 시스템과 장비는 어떻게 설치되고 관리되는가? 답변: 효율적인 시스템을 구축하고 유지하기 위해 다음과 같은 단계를 따를 수 있습니다:

1. 목표 설정: 시스템의 목표를 설정하고 해당 목표를 달성하기 위한 방법을 정의합니다. 예를 들어, 일정 기간마다 자동으로 시스템을 안정화시키거나 자동으로 작업을 수행할 수 있는 기능을 포함할 수 있습니다.

2. 자원 할당: 시스템에 필요한 자원을 할당합니다. 이는 시스템에서 사용되는 장비, 소프트웨어, 네트워크 등을 포함합니다. 가능한 한 많은 자원을 사용하여 시스템을 관리할 수 있도록 합니다.

3. 구성 및 조정: 자원의 할당과 기능을 조정하여 자원의 효율적인 활용을 도모합니다. 이를 위해 자원을 재할당, 프로그램 또는 프로세스 조정 등의 방법을
----------------------------------------------------------------------------------------------------
Response 2: 에너지 효율적인 시스템과 장비는 어떻게 설치되고 관리되는가? 답변: 효율적인 시스템을 구축하고 유지하기 위해서는 다음과 같은 절차를 따를 수 있습니다:

1. 계획: 시스템의 목표와 범위를 설정하고, 필요한 도구와 자원을 할당합니다. 또한, 시스템의 성능을 모니터링하고 최적화하는 단계도 포함됩니다.

2. 설계: 설계된 시스템의 초기 버전을 바탕으로 제품의 기능과 성능을 평가합니다. 이 단계에서는 시스템 구성 요소, 구성 요소 및 리소스, 하드웨어 등을 결정합니다.

3. 자원 할당: 필요한 모든 도구와 장비를 설치합니다. 예를 들어, 컴퓨터, 소프트웨어, 네트워크 장비 등이 필요할 수 있습니다.

4. 설치: 설치된 시스템을 실제로 건설합니다. 실제 시스템에서는 회로
--------------------------------------------------------------------------------------------------

In [ ]:
ranked

[('꽃의 모양이 곤충과의 상호작용에 어떤 영향을 미치는지 설명하시오. 답변: 곤충과 곤충은 서로 다른 종들로, 그들이 상호작용하는 여러 가지 방식이 있습니다. 이러한 상호작용은 곤충에 대한 다음과 같은 영향을 줄 수 있습니다:\n\n1. 교란: 곤충은 곤충에게 교란을 일으킬 수 있습니다. 이는 꽃의 교배나 씨앗 재배 등을 통해 전파될 수 있으며, 곤충들이 다른 꽃이나 식물의 씨앗을 먹거나 꽃의 씨앗이 다른 곳으로 씨앗기를 할 때 발생할 수 있습니다.\n\n2. 산란: 곤충이 곤충들에게 알을 낳으면 산란에 참여합니다. 곤충은 알을 다른 꽃의 성대에 전달하여 꽃의 교배를 도와줍니다. 이렇게 함으로써 곤충들은 꽃의 영양 공급',
  0.20738184452056885),
 ('꽃의 모양이 곤충과의 상호작용에 어떤 영향을 미치는지 설명하시오. 답변: 곤충과 곤충은 서로 다른 종 간 상호작용에서 중요한 역할을 수행합니다.\n\n1. 곤충에 의한 먹이 제공: 많은 곤충은 곤충인 Bioperyron과 같은 작은 곤충을 먹습니다. 이들은 작은 곤충들이나 크기가 작은 벌레, 나비와 같은 곤충들을 먹음으로써 생존합니다.\n도\n2. 곤충 사이의 상호작용: 곤충은 다른 곤충들과 상호작용할 수 있습니다. 예를 들어, 곤충은 꽃의 꿀을 먹은 꿀벌과 상호 작용하여 꽃의 꽃가루를 다른 꽃으로 옮기거나 꽃의 교배를 도웁니다. 이러한 상호작용은 곤충들이 다른 꽃의 성분과 상호작용을 통해 곤충들에게 꽃가루를 옮겨줄',
  0.20738184452056885),
 ('꽃의 모양이 곤충과의 상호작용에 어떤 영향을 미치는지 설명하시오. 답변: 곤충과 곤충은 서로 다른 행동 패턴을 가지고 있습니다. 이러한 상호작용은 곤충에 의해 나타나며, 다양한 형태로 나타날 수 있습니다.\n\n1. 날개: 곤충의 날개는 비행에 필요한 힘을 갖고 있습니다. 날개에는 근육이 있어 비행기를 움직이거나 이동시키는 역할을 합니다. 또한 날개에 있는 작은 근육은 비행 중에 곤충을 잡아내는 역할을 합니다.\n\n2. 부리:

## PPO (Proximal Policy Optimization)

In [ ]:
class PPODataset(Dataset):
    def __init__(self, df, tokenizer, max_length=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        question = self.df.iloc[idx]['question']
        input_text = f"Question: {question}"
        # 토크나이징 후 텐서로 변환
        encoding = self.tokenizer(input_text, truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        return encoding

In [ ]:
PPOdataset = PPODataset(PPOdf, tokenizer)

In [ ]:
QAmodel = GPT2LMHeadModel.from_pretrained('./drive/MyDrive/멋사/챗봇프로젝트/QAmodel50epochs')
tokenizer = PreTrainedTokenizerFast.from_pretrained('./drive/MyDrive/멋사/챗봇프로젝트/QAmodel50epochs')

In [ ]:
Remodel = RewardModel('./drive/MyDrive/멋사/챗봇프로젝트/RMmodel').to(device)

In [ ]:
ppo_config = PPOConfig(batch_size=16, forward_batch_size=4, ppo_epochs=4)
ppo_trainer = PPOTrainer(policy_model, reward_model, tokenizer, config=ppo_config)

# PPO 학습 루프
for epoch in range(num_epochs):
    for question in dataset:
        query_tensors = tokenizer(question, return_tensors="pt").input_ids
        response_tensors = policy_model.generate(query_tensors)

        # 리워드 계산
        rewards = reward_model(input_ids=response_tensors)

        # PPO 업데이트
        ppo_trainer.step(query_tensors, response_tensors, rewards)